In [1]:
import numpy as np

def calculate_bollinger_bands(data, period=20, deviation=2):
    """
    Calculate Bollinger Bands for a given dataset.

    Parameters:
        data (list or numpy.array): List of closing prices.
        period (int): Period for the moving average and standard deviation.
        deviation (float): Number of standard deviations for the bands.

    Returns:
        upper_band (numpy.array): Upper Bollinger Band values.
        middle_band (numpy.array): Middle Bollinger Band (SMA) values.
        lower_band (numpy.array): Lower Bollinger Band values.
    """
    close_prices = np.array(data)
    sma = np.mean(close_prices[-period:])
    std = np.std(close_prices[-period:])

    upper_band = sma + (deviation * std)
    lower_band = sma - (deviation * std)

    return upper_band, sma, lower_band


In [2]:
import numpy as np

def calculate_average_true_range(high, low, close, period=14):
    """
    Calculate Average True Range (ATR) for a given dataset.

    Parameters:
        high (list or numpy.array): List of high prices.
        low (list or numpy.array): List of low prices.
        close (list or numpy.array): List of closing prices.
        period (int): Period for calculating ATR.

    Returns:
        atr (numpy.array): Average True Range values.
    """
    high_prices = np.array(high)
    low_prices = np.array(low)
    close_prices = np.array(close)

    tr = np.maximum(high_prices - low_prices, np.abs(high_prices - close_prices[1:]), np.abs(low_prices - close_prices[1:]))
    atr = np.zeros_like(close_prices)

    atr[period] = np.mean(tr[:period])

    for i in range(period + 1, len(atr)):
        atr[i] = (atr[i - 1] * (period - 1) + tr[i]) / period

    return atr


In [3]:
import numpy as np

def calculate_percentage_price_oscillator(data, short_period=12, long_period=26, signal_period=9):
    """
    Calculate Percentage Price Oscillator (PPO) for a given dataset.

    Parameters:
        data (list or numpy.array): List of closing prices.
        short_period (int): Short-term EMA period.
        long_period (int): Long-term EMA period.
        signal_period (int): Signal line EMA period.

    Returns:
        ppo (numpy.array): Percentage Price Oscillator values.
        signal_line (numpy.array): Signal line values.
    """
    close_prices = np.array(data)

    short_ema = np.mean(close_prices[-short_period:])
    long_ema = np.mean(close_prices[-long_period:])

    ppo = ((short_ema - long_ema) / long_ema) * 100

    signal_line = np.mean(ppo[-signal_period:])

    return ppo, signal_line


In [4]:
import numpy as np

def calculate_kama(data, period=10, fast_period=2, slow_period=30):
    """
    Calculate Kaufman's Adaptive Moving Average (KAMA) for a given dataset.

    Parameters:
        data (list or numpy.array): List of closing prices.
        period (int): Smoothing period.
        fast_period (int): Fast EMA period for efficiency ratio calculation.
        slow_period (int): Slow EMA period for efficiency ratio calculation.

    Returns:
        kama (numpy.array): Kaufman's Adaptive Moving Average values.
    """
    close_prices = np.array(data)

    er = np.abs(close_prices - np.roll(close_prices, period))
    fast_alpha = 2 / (fast_period + 1)
    slow_alpha = 2 / (slow_period + 1)

    fast_er_sum = np.sum(er[-fast_period:])
    slow_er_sum = np.sum(er[-slow_period:])

    efficiency_ratio = fast_er_sum / slow_er_sum
    smoothing_const = (efficiency_ratio * (fast_alpha - slow_alpha) + slow_alpha) ** 2

    kama = np.zeros_like(close_prices)

    kama[period - 1] = close_prices[period - 1]

    for i in range(period, len(kama)):
        kama[i] = kama[i - 1] + smoothing_const * (close_prices[i] - kama[i - 1])

    return kama


In [5]:
import numpy as np

def calculate_cci(high, low, close, period=14):
    """
    Calculate Commodity Channel Index (CCI) for a given dataset.

    Parameters:
        high (list or numpy.array): List of high prices.
        low (list or numpy.array): List of low prices.
        close (list or numpy.array): List of closing prices.
        period (int): Period for calculating CCI.

    Returns:
        cci (numpy.array): Commodity Channel Index values.
    """
    high_prices = np.array(high)
    low_prices = np.array(low)
    close_prices = np.array(close)

    typical_prices = (high_prices + low_prices + close_prices) / 3
    mean_deviation = np.zeros_like(close_prices)

    for i in range(period - 1, len(close_prices)):
        typical_prices_slice = typical_prices[i - period + 1:i + 1]
        mean_deviation[i] = np.mean(np.abs(typical_prices[i] - typical_prices_slice))

    mean_deviation[mean_deviation == 0] = 0.0001  # To avoid division by zero

    cci = (typical_prices - np.mean(typical_prices)) / (0.015 * mean_deviation)

    return cci


In [6]:
import numpy as np

def calculate_mass_index(high, low, period=25, ema_period=9):
    """
    Calculate Mass Index for a given dataset.

    Parameters:
        high (list or numpy.array): List of high prices.
        low (list or numpy.array): List of low prices.
        period (int): Period for calculating Mass Index.
        ema_period (int): Period for calculating Exponential Moving Average (EMA) of Mass Index.

    Returns:
        mass_index (numpy.array): Mass Index values.
    """
    high_prices = np.array(high)
    low_prices = np.array(low)

    range_high_low = high_prices - low_prices
    ex_range = np.zeros_like(range_high_low)

    for i in range(1, len(ex_range)):
        ex_range[i] = np.sum(range_high_low[i - period + 1:i + 1])

    single_ema = ex_range / period
    double_ema = np.zeros_like(single_ema)

    for i in range(1, len(double_ema)):
        double_ema[i] = np.sum(single_ema[i - ema_period + 1:i + 1])

    mass_index = np.zeros_like(double_ema)

    for i in range(period, len(mass_index)):
        mass_index[i] = double_ema[i] / single_ema[i]

    return mass_index


In [7]:
import numpy as np

def calculate_stc(close, period=10, short_cycle=23, long_cycle=50, smoothing=3):
    """
    Calculate Schaff Trend Cycle (STC) for a given dataset.

    Parameters:
        close (list or numpy.array): List of closing prices.
        period (int): Period for %K calculation.
        short_cycle (int): Short-term cycle period.
        long_cycle (int): Long-term cycle period.
        smoothing (int): Smoothing period.

    Returns:
        stc (numpy.array): Schaff Trend Cycle values.
    """
    close_prices = np.array(close)

    high_high = np.max(close_prices[-short_cycle:])
    low_low = np.min(close_prices[-short_cycle:])

    raw_k = ((close_prices[-1] - low_low) / (high_high - low_low)) * 100
    k = np.zeros_like(close_prices)

    k[period - 1] = raw_k

    for i in range(period, len(k)):
        k[i] = (raw_k + (k[i - 1] * (period - 1))) / period

    d = np.zeros_like(k)

    d[period - 1] = np.mean(k[period - smoothing:period])

    for i in range(period, len(d)):
        d[i] = np.mean(k[i - smoothing:i])

    raw_stc = ((k[-1] + 2 * d[-1]) / 3) * 100
    stc = np.zeros_like(close_prices)

    stc[short_cycle - 1] = raw_stc

    for i in range(short_cycle, len(stc)):
        raw_k = ((close_prices[i - short_cycle] - np.min(close_prices[i - short_cycle:i])) / (np.max(close_prices[i - short_cycle:i]) - np.min(close_prices[i - short_cycle:i]))) * 100
        k[i] = (raw_k + (k[i - 1] * (period - 1))) / period
        d[i] = np.mean(k[i - smoothing:i])
        stc[i] = ((k[i] + 2 * d[i]) / 3) * 100

    return stc


In [8]:
import numpy as np

def calculate_vortex_indicator_pos(high, low, close, period=14):
    """
    Calculate Vortex Indicator (Positive) for a given dataset.

    Parameters:
        high (list or numpy.array): List of high prices.
        low (list or numpy.array): List of low prices.
        close (list or numpy.array): List of closing prices.
        period (int): Period for calculating the indicator.

    Returns:
        vortex_pos (numpy.array): Vortex Indicator (Positive) values.
    """
    high_prices = np.array(high)
    low_prices = np.array(low)
    close_prices = np.array(close)

    tr = np.maximum(high_prices - low_prices, np.abs(high_prices - close_prices[1:]), np.abs(low_prices - close_prices[1:]))
    vm_plus = np.abs(high_prices[:-1] - low_prices[1:])

    vortex_pos = np.zeros_like(close_prices)

    for i in range(period - 1, len(vortex_pos)):
        tr_sum = np.sum(tr[i - period + 1:i + 1])
        vm_plus_sum = np.sum(vm_plus[i - period + 1:i + 1])
        vortex_pos[i] = (vm_plus_sum / tr_sum) * 100 if tr_sum != 0 else 0

    return vortex_pos


In [9]:
import numpy as np

def calculate_vortex_indicator_neg(high, low, close, period=14):
    """
    Calculate Vortex Indicator (Negative) for a given dataset.

    Parameters:
        high (list or numpy.array): List of high prices.
        low (list or numpy.array): List of low prices.
        close (list or numpy.array): List of closing prices.
        period (int): Period for calculating the indicator.

    Returns:
        vortex_neg (numpy.array): Vortex Indicator (Negative) values.
    """
    high_prices = np.array(high)
    low_prices = np.array(low)
    close_prices = np.array(close)

    tr = np.maximum(high_prices - low_prices, np.abs(high_prices - close_prices[1:]), np.abs(low_prices - close_prices[1:]))
    vm_minus = np.abs(low_prices[:-1] - high_prices[1:])

    vortex_neg = np.zeros_like(close_prices)

    for i in range(period - 1, len(vortex_neg)):
        tr_sum = np.sum(tr[i - period + 1:i + 1])
        vm_minus_sum = np.sum(vm_minus[i - period + 1:i + 1])
        vortex_neg[i] = (vm_minus_sum / tr_sum) * 100 if tr_sum != 0 else 0

    return vortex_neg


In [10]:
import numpy as np

def calculate_parabolic_sar_down(high, low, af_start=0.02, af_increment=0.02, af_max=0.2):
    """
    Calculate Parabolic SAR (SAR Down) for a given dataset.

    Parameters:
        high (list or numpy.array): List of high prices.
        low (list or numpy.array): List of low prices.
        af_start (float): Initial acceleration factor.
        af_increment (float): Increment for acceleration factor.
        af_max (float): Maximum acceleration factor.

    Returns:
        sar_down (numpy.array): Parabolic SAR (SAR Down) values.
    """
    high_prices = np.array(high)
    low_prices = np.array(low)

    af = af_start
    sar = np.zeros_like(high_prices)
    ep = low_prices[0]
    trend = -1  # Initial trend is down

    for i in range(2, len(high_prices)):
        if trend == -1:
            sar[i] = sar[i - 1] - af * (sar[i - 1] - ep)
        else:
            sar[i] = sar[i - 1] + af * (ep - sar[i - 1])

        if trend == -1 and high_prices[i] > high_prices[i - 1]:
            trend = 1
            sar[i] = ep
            ep = high_prices[i]
            af = af_start
        elif trend == 1 and low_prices[i] < low_prices[i - 1]:
            trend = -1
            sar[i] = ep
            ep = low_prices[i]
            af = af_start
        elif trend == 1:
            if high_prices[i] > ep:
                ep = high_prices[i]
                af = min(af + af_increment, af_max)
        elif trend == -1:
            if low_prices[i] < ep:
                ep = low_prices[i]
                af = min(af + af_increment, af_max)

    return sar


In [11]:
import numpy as np

def calculate_ichimoku_indicator(high, low, tenkan_sen_period=9, kijun_sen_period=26, senkou_span_b_period=52, displacement=26):
    """
    Calculate Ichimoku Indicator for a given dataset.

    Parameters:
        high (list or numpy.array): List of high prices.
        low (list or numpy.array): List of low prices.
        tenkan_sen_period (int): Tenkan-sen period.
        kijun_sen_period (int): Kijun-sen period.
        senkou_span_b_period (int): Senkou Span B period.
        displacement (int): Displacement value.

    Returns:
        tenkan_sen (numpy.array): Tenkan-sen values.
        kijun_sen (numpy.array): Kijun-sen values.
        senkou_span_a (numpy.array): Senkou Span A values.
        senkou_span_b (numpy.array): Senkou Span B values.
    """
    high_prices = np.array(high)
    low_prices = np.array(low)

    tenkan_sen = (np.max(high_prices[-tenkan_sen_period:]) + np.min(low_prices[-tenkan_sen_period:])) / 2
    kijun_sen = (np.max(high_prices[-kijun_sen_period:]) + np.min(low_prices[-kijun_sen_period:])) / 2

    senkou_span_a = ((tenkan_sen + kijun_sen) / 2)

    high52 = np.max(high_prices[-senkou_span_b_period:])
    low52 = np.min(low_prices[-senkou_span_b_period:])
    senkou_span_b = ((high52 + low52) / 2)

    return tenkan_sen, kijun_sen, senkou_span_a, senkou_span_b


In [12]:
import numpy as np

def calculate_ema_indicator(data, period=14):
    """
    Calculate Exponential Moving Average (EMA) Indicator for a given dataset.

    Parameters:
        data (list or numpy.array): List of closing prices.
        period (int): Period for calculating EMA.

    Returns:
        ema (numpy.array): Exponential Moving Average (EMA) values.
    """
    close_prices = np.array(data)

    ema = np.zeros_like(close_prices)
    multiplier = 2 / (period + 1)

    ema[period - 1] = np.mean(close_prices[:period])

    for i in range(period, len(ema)):
        ema[i] = (close_prices[i] - ema[i - 1]) * multiplier + ema[i - 1]

    return ema


In [13]:
import numpy as np

def calculate_ichimoku_a(high, low, tenkan_sen_period=9, kijun_sen_period=26, senkou_span_b_period=52, displacement=26):
    """
    Calculate Ichimoku Senkou Span A for a given dataset.

    Parameters:
        high (list or numpy.array): List of high prices.
        low (list or numpy.array): List of low prices.
        tenkan_sen_period (int): Tenkan-sen period.
        kijun_sen_period (int): Kijun-sen period.
        senkou_span_b_period (int): Senkou Span B period.
        displacement (int): Displacement value.

    Returns:
        senkou_span_a (numpy.array): Ichimoku Senkou Span A values.
    """
    tenkan_sen, kijun_sen, senkou_span_a, senkou_span_b = calculate_ichimoku_indicator(high, low, tenkan_sen_period, kijun_sen_period, senkou_span_b_period, displacement)

    return senkou_span_a


In [14]:
import numpy as np

def calculate_ichimoku_b(high, low, tenkan_sen_period=9, kijun_sen_period=26, senkou_span_b_period=52, displacement=26):
    """
    Calculate Ichimoku Senkou Span B for a given dataset.

    Parameters:
        high (list or numpy.array): List of high prices.
        low (list or numpy.array): List of low prices.
        tenkan_sen_period (int): Tenkan-sen period.
        kijun_sen_period (int): Kijun-sen period.
        senkou_span_b_period (int): Senkou Span B period.
        displacement (int): Displacement value.

    Returns:
        senkou_span_b (numpy.array): Ichimoku Senkou Span B values.
    """
    tenkan_sen, kijun_sen, senkou_span_a, senkou_span_b = calculate_ichimoku_indicator(high, low, tenkan_sen_period, kijun_sen_period, senkou_span_b_period, displacement)

    return senkou_span_b


In [15]:
import numpy as np

def calculate_ichimoku_baseline(high, low, kijun_sen_period=26):
    """
    Calculate Ichimoku Baseline (Kijun-sen) for a given dataset.

    Parameters:
        high (list or numpy.array): List of high prices.
        low (list or numpy.array): List of low prices.
        kijun_sen_period (int): Kijun-sen period.

    Returns:
        kijun_sen (numpy.array): Ichimoku Baseline (Kijun-sen) values.
    """
    kijun_sen = (np.max(high[-kijun_sen_period:]) + np.min(low[-kijun_sen_period:])) / 2

    return kijun_sen


In [ ]:
import refinitiv.data as rd
import time
import pandas as pd

# Replace with your App Key and Secret
app_key = 'YOUR_APP_KEY'
app_secret = 'YOUR_APP_SECRET'

# Define the instrument and service name
instrument = 'USDJPY='
service_name = '/v1/pricing/streaming/prices'

# Create an RDP session
session = rd.Session(app_key, app_secret)
session.open()

price_data = []  # Initialize an empty list to store price data

try:
    # Create a streaming request
    request = rd.StreamingPricesRequest(service_name)
    request.set_query("Space", "NewsEquityOptionSpace")
    request.set_query("Interval", "300s")  # 5 seconds interval

    # Add instruments to the request
    request.add_instrument(instrument)

    while True:
        # Open a streaming session
        session.open_streaming_session(request)

        # Receive and collect streaming prices
        for msg in session:
            if msg.is_data():
                fields = msg.get_fields()
                bid_price = fields.get_double("BID")
                ask_price = fields.get_double("ASK")
                timestamp = fields.get_time("ACTIV_DATE")

                price_data.append({
                    'Timestamp': timestamp,
                    'Bid Price': bid_price,
                    'Ask Price': ask_price
                })

        # Close the streaming session
        session.close_streaming_session()

        # Save data to a CSV file periodically (e.g., every minute)
        if len(price_data) % 12 == 0:  # Save every minute (12 * 5 seconds = 1 minute)
            df = pd.DataFrame(price_data)
            df.to_csv('price_data.csv', index=False)

        # Wait for 5 seconds before the next request
        time.sleep(5)

except Exception as e:
    print(f'Error: {e}')

finally:
    # Close the RDP session
    session.close()
